In [2]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np
from skimage.transform import resize

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from data_manager import *
from metadata import *
from conversion import images_to_volume

rm -rf "/shared/CSHL_data_processed/MD662/MD662_sorted_filenames.txt" && mkdir -p "/shared/CSHL_data_processed/MD662"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_sorted_filenames.txt" "/shared/CSHL_data_processed/MD662/MD662_sorted_filenames.txt"
rm -rf "/shared/CSHL_data_processed/MD662/MD662_anchor.txt" && mkdir -p "/shared/CSHL_data_processed/MD662"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_anchor.txt" "/shared/CSHL_data_processed/MD662/MD662_anchor.txt"
rm -rf "/shared/CSHL_data_processed/MD662/MD662_anchor.txt" && mkdir -p "/shared/CSHL_data_processed/MD662"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_anchor.txt" "/shared/CSHL_data_processed/MD662/MD662_anchor.txt"


File does not exist: /shared/CSHL_data_processed/MD662/MD662_anchor.txt


rm -rf "/shared/CSHL_data_processed/MD661/MD661_thumbnail_alignedTo_MD662&661-F116-2017.06.07-04.39.41_MD661_1_0346_cropped/MD662&661-F4-2017.06.02-18.04.01_MD661_1_0010_thumbnail_alignedTo_MD662&661-F116-2017.06.07-04.39.41_MD661_1_0346_cropped.tif" && mkdir -p "/shared/CSHL_data_processed/MD661/MD661_thumbnail_alignedTo_MD662&661-F116-2017.06.07-04.39.41_MD661_1_0346_cropped"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD661/MD661_thumbnail_alignedTo_MD662&661-F116-2017.06.07-04.39.41_MD661_1_0346_cropped/MD662&661-F4-2017.06.02-18.04.01_MD661_1_0010_thumbnail_alignedTo_MD662&661-F116-2017.06.07-04.39.41_MD661_1_0346_cropped.tif" "/shared/CSHL_data_processed/MD661/MD661_thumbnail_alignedTo_MD662&661-F116-2017.06.07-04.39.41_MD661_1_0346_cropped/MD662&661-F4-2017.06.02-18.04.01_MD661_1_0010_thumbnail_alignedTo_MD662&661-F116-2017.06.07-04.39.41_MD661_1_0346_cropped.tif"
rm -rf "/shared/CSHL_data_processed/MD661/MD661_thumbnail_alignedTo_MD662&661-F116-2017.06.07-04.39.41_

rm -rf "/shared/CSHL_data_processed/MD662/MD662_anchor.txt" && mkdir -p "/shared/CSHL_data_processed/MD662"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_anchor.txt" "/shared/CSHL_data_processed/MD662/MD662_anchor.txt"


File does not exist: /shared/CSHL_data_processed/MD662/MD662_anchor.txt


...

File does not exist: /shared/CSHL_data_processed/MD662/MD662_sorted_filenames.txt


rm -rf "/shared/CSHL_data_processed/MD662/MD662_anchor.txt" && mkdir -p "/shared/CSHL_data_processed/MD662"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_anchor.txt" "/shared/CSHL_data_processed/MD662/MD662_anchor.txt"


File does not exist: /shared/CSHL_data_processed/MD662/MD662_anchor.txt


rm -rf "/shared/CSHL_data_processed/MD662/MD662_anchor.txt" && mkdir -p "/shared/CSHL_data_processed/MD662"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_anchor.txt" "/shared/CSHL_data_processed/MD662/MD662_anchor.txt"
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


File does not exist: /shared/CSHL_data_processed/MD662/MD662_anchor.txt


In [2]:
# downsample = 1
# downsample = 32
# downsample = 8
# downsample=8 is the maximum. Exceeding it results in memory error.

In [5]:
# for stack in all_nissl_stacks:
# for stack in ['MD652', 'MD653', 'MD657', 'MD658']:
for stack in ['MD661']:
    
    first_sec, last_sec = metadata_cache['section_limits'][stack]
    
#     for downsample in [8, 32]:
    for downsample in [32]:

        images = {}
        for sec in metadata_cache['valid_sections'][stack]:
            print sec
            mask = DataManager.load_thumbnail_mask_v3(stack=stack, section=sec, version='aligned_cropped')

            if downsample == 32:
                img_rgb = DataManager.load_image(stack, section=sec, resol='thumbnail', version='cropped_tif')
                img = img_as_ubyte(rgb2gray(img_rgb))
                img[~mask] = 0
            else:
                img_rgb = DataManager.load_image(stack, section=sec, resol='lossless', version='cropped_gray')
                if downsample == 1:
                    img = img_as_ubyte(rgb2gray(img_rgb)) 
                else:
                    img = img_as_ubyte(rgb2gray(img_rgb[::downsample, ::downsample]))
                    mask = resize(mask, img.shape[:2]).astype(np.bool)
                img[~mask] = 0

            images[sec-1] = img

        voxel_z_size = SECTION_THICKNESS/(XY_PIXEL_DISTANCE_LOSSLESS * downsample)

        volume, volume_bbox = images_to_volume(images=images, voxel_size=(1, 1, voxel_z_size), first_sec=first_sec-1, last_sec=last_sec-1)
    #     volume, volume_bbox = images_to_volume(images=images, voxel_size=(1, 1, voxel_z_size), first_sec=first_sec-1, last_sec=first_sec-1+10)
        print volume.shape

        vol_fp = DataManager.get_intensity_volume_filepath(stack=stack, downscale=downsample)
        create_if_not_exists(os.path.dirname(vol_fp))
        bp.pack_ndarray_file(volume, vol_fp)
        upload_to_s3(vol_fp)

        bbox_fp = DataManager.get_intensity_volume_bbox_filepath(stack=stack, downscale=downsample)
        np.savetxt(bbox_fp, np.array(volume_bbox, np.int)[None, :], fmt='%d')
        upload_to_s3(bbox_fp)

90
rm -rf /shared/CSHL_data_processed/MD652/MD652_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_masks_cropped/MD652-F15-2016.12.16-20.45.52_MD652_3_0045_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_mask_cropped.png && mkdir -p /shared/CSHL_data_processed/MD652/MD652_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_masks_cropped
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD652/MD652_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_masks_cropped/MD652-F15-2016.12.16-20.45.52_MD652_3_0045_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_mask_cropped.png /shared/CSHL_data_processed/MD652/MD652_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_masks_cropped/MD652-F15-2016.12.16-20.45.52_MD652_3_0045_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_mask_cropped.png


Child returned 0
Child returned 0
0.59 seconds.
Child returned 0


rm -rf /shared/CSHL_data_processed/MD652/MD652_lossless_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_cropped_gray/MD652-F15-2016.12.16-20.45.52_MD652_3_0045_lossless_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_cropped_gray.tif && mkdir -p /shared/CSHL_data_processed/MD652/MD652_lossless_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_cropped_gray
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD652/MD652_lossless_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_cropped_gray/MD652-F15-2016.12.16-20.45.52_MD652_3_0045_lossless_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_cropped_gray.tif /shared/CSHL_data_processed/MD652/MD652_lossless_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_cropped_gray/MD652-F15-2016.12.16-20.45.52_MD652_3_0045_lossless_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_cropped_gray.tif


Child returned 0
1.65 seconds.


91
rm -rf /shared/CSHL_data_processed/MD652/MD652_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_masks_cropped/MD652-N16-2016.12.16-06.00.10_MD652_1_0046_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_mask_cropped.png && mkdir -p /shared/CSHL_data_processed/MD652/MD652_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_masks_cropped
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD652/MD652_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_masks_cropped/MD652-N16-2016.12.16-06.00.10_MD652_1_0046_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_mask_cropped.png /shared/CSHL_data_processed/MD652/MD652_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_masks_cropped/MD652-N16-2016.12.16-06.00.10_MD652_1_0046_alignedTo_MD652-F45-2016.12.17-05.56.31_MD652_2_0134_mask_cropped.png


...

Child returned 0
0.73 seconds.


104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
254
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
318
319
320
321
322
323
324
325
326
327
328
329
330
331
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358


Child returned 0
0.99 seconds.


aws s3 cp /shared/CSHL_volumes/MD658/MD658_down32_intensityVolume/MD658_down32_intensityVolume_bbox.txt s3://mousebrainatlas-data/CSHL_volumes/MD658/MD658_down32_intensityVolume/MD658_down32_intensityVolume_bbox.txt


Child returned 0
0.41 seconds.


In [5]:
display_image(volume[200,:,:])

/shared/MouseBrainAtlas/reconstruct/tmp.jpg